# Assignment 3

Welcome to Assignment 3. This will be even more fun. Now we will calculate statistical measures on the test data you have created.

YOU ARE NOT ALLOWED TO USE ANY OTHER 3RD PARTY LIBRARIES LIKE PANDAS. PLEASE ONLY MODIFY CONTENT INSIDE THE FUNCTION SKELETONS
Please read why: https://www.coursera.org/learn/exploring-visualizing-iot-data/discussions/weeks/3/threads/skjCbNgeEeapeQ5W6suLkA
. Just make sure you hit the play button on each cell from top to down. There are seven functions you have to implement. Please also make sure than on each change on a function you hit the play button again on the corresponding cell to make it available to the rest of this notebook.
Please also make sure to only implement the function bodies and DON'T add any additional code outside functions since this might confuse the autograder.

So the function below is used to make it easy for you to create a data frame from a cloudant data frame using the so called "DataSource" which is some sort of a plugin which allows ApacheSpark to use different data sources.


In [1]:
#Please don't modify this function
def readDataFrameFromCloudant(database):
    cloudantdata=spark.read.load(database, "org.apache.bahir.cloudant")

    cloudantdata.createOrReplaceTempView("washing")
    spark.sql("SELECT * from washing").show()
    return cloudantdata

All functions can be implemented using DataFrames, ApacheSparkSQL or RDDs. We are only interested in the result. You are given the reference to the data frame in the "df" parameter and in case you want to use SQL just use the "spark" parameter which is a reference to the global SparkSession object. Finally if you want to use RDDs just use "df.rdd" for obtaining a reference to the underlying RDD object. 

Let's start with the first function. Please calculate the minimal temperature for the test data set you have created. We've provided a little skeleton for you in case you want to use SQL. You can use this skeleton for all subsequent functions. Everything can be implemented using SQL only if you like.

In [2]:
def minTemperature(df,spark):
    return spark.sql("SELECT min(temperature) as mintemp from washing").first().mintemp

Please now do the same for the mean of the temperature

In [3]:
def meanTemperature(df,spark):
    return spark.sql("SELECT avg(temperature) as avg_temperature from washing WHERE temperature is not null").first().avg_temperature

Please now do the same for the maximum of the temperature

In [4]:
def maxTemperature(df,spark):
    return spark.sql("SELECT max(temperature) as maxtemp from washing").first().maxtemp

Please now do the same for the standard deviation of the temperature

In [99]:
def sdTemperature(df,spark):
    tempData = spark.sql("SELECT temperature FROM washing WHERE temperature is not null").rdd.map(list).flatMap(lambda x : x)
    return tempData.stdev()

Please now do the same for the skew of the temperature. Since the SQL statement for this is a bit more complicated we've provided a skeleton for you. You have to insert custom code at four position in order to make the function work. Alternatively you can also remove everything and implement if on your own. Note that we are making use of two previously defined functions, so please make sure they are correct. Also note that we are making use of python's string formatting capabilitis where the results of the two function calls to "meanTemperature" and "sdTemperature" are inserted at the "%s" symbols in the SQL string.

In [124]:
def skewTemperature(df,spark):
    tempData = spark.sql("SELECT temperature FROM washing WHERE temperature is not null").rdd.map(list).flatMap(lambda x : x)
    numTemp = float(tempData.count())
    meanTemp = meanTemperature(df, spark)
    stdevTemp = sdTemperature(df, spark)
    skewTemp = tempData.map(lambda x : pow(x - meanTemp, 3) / pow(stdevTemp, 3)).sum() / numTemp
    return skewTemp

Kurtosis is the 4th statistical moment, so if you are smart you can make use of the code for skew which is the 3rd statistical moment. Actually only two things are different.

In [126]:
def kurtosisTemperature(df,spark): 
    tempData = spark.sql("SELECT temperature FROM washing WHERE temperature is not null").rdd.map(list).flatMap(lambda x : x)
    numTemp = float(tempData.count())
    meanTemp = meanTemperature(df, spark)
    stdTemp = sdTemperature(df, spark)
    kurtTemp = tempData.map(lambda x : pow(x - meanTemp, 4) / pow(stdTemp, 4)).sum() / numTemp
    return kurtTemp

Just a hint. This can be solved easily using SQL as well, but as shown in the lecture also using RDDs.

In [127]:
def correlationTemperatureHardness(df,spark):
    return df.corr("temperature", "hardness")

### PLEASE DON'T REMOVE THIS BLOCK - THE FOLLOWING CODE IS NOT GRADED
#axx
### PLEASE DON'T REMOVE THIS BLOCK - THE FOLLOWING CODE IS NOT GRADED

Now it is time to connect to the cloudant database. Please have a look at the Video "Overview of end-to-end scenario" of Week 2 starting from 6:40 in order to learn how to obtain the credentials for the database. Please paste this credentials as strings into the below code

### TODO Please provide your Cloudant credentials here

In [9]:
hostname = "1783f19f-3912-4211-893e-b341edeeffd4-bluemix.cloudant.com"
user = "1783f19f-3912-4211-893e-b341edeeffd4-bluemix"
pw = "1f30ff869624227b320f148c002daacb75c8ff001ec4ef2f3947cb1de53624a2"
database = "washing" #as long as you didn't change this in the NodeRED flow the database name stays the same

In [10]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",hostname)\
    .config("cloudant.username", user)\
    .config("cloudant.password",pw)\
    .getOrCreate()
cloudantdata=readDataFrameFromCloudant(database)

+--------------------+--------------------+------+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev| count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+------+--------+----------+---------+--------+-----+-----------+-------------+-------+
|016f908726846e142...|1-b54c0604b68175d...|129627|      11|acceptable|     null|      70| null|         84|1534638608182|   null|
|016f908726846e142...|1-80b4337ac969602...|129637|      11|acceptable|     null|      70| null|         83|1534638618235|   null|
|016f908726846e142...|1-1d5ae4523fe5f32...|129640|      11|acceptable|     null|      79| null|         81|1534638621245|   null|
|016f908726846e142...|1-b40c8ffe9d1c653...| 26128|    null|      null|     null|    null| 1046|       null|1534638642224|   null|
|016f908726846e142...|1-299444d07544b7e...|129664|      11|acceptable|     null|      80| 

In [11]:
minTemperature(cloudantdata,spark)

80

In [12]:
meanTemperature(cloudantdata,spark)

89.99873002761932

In [13]:
maxTemperature(cloudantdata,spark)

100

In [100]:
sdTemperature(cloudantdata,spark)

6.0440121437560963

In [125]:
skewTemperature(cloudantdata,spark)

-0.0013215271960618198

In [128]:
kurtosisTemperature(cloudantdata,spark)

1.798537149542315

In [129]:
correlationTemperatureHardness(cloudantdata,spark)

0.9198633186598345

Congratulations, you are done, please download this notebook as python file using the export function and submit is to the gader using the filename "assignment3.1.py"